# Sistema de Busca Semântica para Portarias da UDESC

Este notebook apresenta um sistema de busca inteligente baseado em modelos de linguagem (LLMs) e indexação vetorial com FAISS. Ele permite a recuperação de documentos administrativos (portarias e resoluções) da UDESC com base em similaridade semântica e textual.


In [20]:
#Instalação de dependências (caso não estejam disponíveis)
!pip install sentence-transformers faiss-cpu unidecode PyMuPDF
!pip install pymupdf
!pip install unidecode

In [5]:
# Preparação dos dados (opcional)

import fitz
import os
import re

# Criar pasta para os textos extraídos
os.makedirs("textos_extraidos", exist_ok=True)

# Caminho dos PDFs
pasta_pdfs = "portarias"

# Lista de arquivos que você quer ignorar por enquanto
ignorar = [
    "Documento_de_área_2019.pdf",
    "Documento_de_área_2017.pdf",
    "Documento_de_área_2013_-_Regras_do_CA-CC_para_Avaliação_de_Mestrados_E_Qualis_Periódicos_(WebQualis).pdf",
    "Documento_de_área_2013_-_Regras_do_CA-CC_para_Avaliação_de_Mestrados.pdf"
]

# Função para extrair texto de um PDF
def extrair_texto_pdf(caminho_pdf):
    doc = fitz.open(caminho_pdf)
    texto_total = ""
    for pagina in doc:
        texto_total += pagina.get_text()
    doc.close()
    return texto_total

# Processar os PDFs válidos
erros = []

for arquivo in os.listdir(pasta_pdfs):
    if arquivo.endswith(".pdf") and arquivo not in ignorar:
        caminho = os.path.join(pasta_pdfs, arquivo)
        try:
            texto = extrair_texto_pdf(caminho)
            nome_txt = arquivo.replace(".pdf", ".txt")
            with open(os.path.join("textos_extraidos", nome_txt), "w", encoding="utf-8") as f:
                f.write(texto)
        except Exception as e:
            print(f"Erro ao processar {arquivo}: {e}")
            erros.append(arquivo)

# Mostrar quais arquivos deram erro
if erros:
    print("\n⚠️ Arquivos com erro de leitura:")
    for erro in erros:
        print(f"- {erro}")
else:
    print("\n✅ Todos os arquivos foram processados com sucesso!")



✅ Todos os arquivos foram processados com sucesso!


In [7]:
#Geração do índice FAISS

from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import os
import pickle

modelo = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

textos = []
nomes = []

for nome_arquivo in os.listdir("textos_extraidos"):
    if nome_arquivo.endswith(".txt"):
        with open(os.path.join("textos_extraidos", nome_arquivo), "r", encoding="utf-8") as f:
            textos.append(f.read())
        nomes.append(nome_arquivo)

embeddings = modelo.encode(textos, convert_to_numpy=True)
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

faiss.write_index(index, "index_faiss.idx")
with open("nomes_textos.pkl", "wb") as f:
    pickle.dump(nomes, f)



In [11]:
# Sistema de busca interativo (literal + semântica)

import os
import faiss
import pickle
import numpy as np
import re
import json
import textwrap
from sentence_transformers import SentenceTransformer
from unidecode import unidecode
from IPython.display import Markdown, display

# Carregar modelo e índice
modelo = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
index = faiss.read_index("index_faiss.idx")

with open("nomes_textos.pkl", "rb") as f:
    nomes = pickle.load(f)

# Limpar nomes inválidos
nomes = [nome.strip() for nome in nomes if nome.strip() and nome != ".txt" and nome != "​.txt"]

# Carregar links mapeados
with open("mapeamento_links.json", "r", encoding="utf-8") as f:
    mapeamento_links = json.load(f)

# 🔎 Função robusta para encontrar o link correto do PDF
def encontrar_link(nome_arquivo):
    def normalizar(texto):
        texto = unidecode(texto.lower())
        texto = re.sub(r'[\W_]+', '', texto)  # Remove tudo que não é letra ou número
        return texto

    chave_normalizada = normalizar(nome_arquivo)

    # Primeiro: busca exata com normalização
    for chave in mapeamento_links:
        if normalizar(chave) == chave_normalizada:
            return mapeamento_links[chave]

    # Segundo: busca por inclusão parcial
    for chave in mapeamento_links:
        if chave_normalizada in normalizar(chave):
            return mapeamento_links[chave]

    return None

# 📘 Busca literal nos textos
def busca_literal_em_todos(pergunta):
    resultados = []
    pergunta_lower = pergunta.lower().strip()

    for nome in nomes:
        caminho = os.path.join("textos_extraidos", nome)
        if not os.path.exists(caminho):
            continue

        with open(caminho, "r", encoding="utf-8") as f:
            conteudo = f.read()

        if pergunta_lower in conteudo.lower():
            for linha in conteudo.splitlines():
                if pergunta_lower in linha.lower():
                    trecho = linha.strip()
                    break
            else:
                trecho = textwrap.shorten(conteudo.replace("\n", " "), width=300, placeholder=" [...]")

            link = encontrar_link(nome)

            resultados.append({
                "nome": nome,
                "trecho": trecho,
                "link": link,
                "similaridade": None
            })

    return resultados

# 🤖 Busca semântica com fallback
def busca_semantica(pergunta, top_k=3):
    embedding = modelo.encode([pergunta], convert_to_numpy=True)
    distancias, indices = index.search(embedding, top_k)

    resultados = []
    for idx in indices[0]:
        if idx >= len(nomes):
            continue
        nome = nomes[idx]
        caminho = os.path.join("textos_extraidos", nome)
        if not os.path.exists(caminho):
            continue

        with open(caminho, "r", encoding="utf-8") as f:
            conteudo = f.read()

        # Tenta achar trecho com palavras da pergunta
        pergunta_lower = pergunta.lower()
        trecho_encontrado = ""
        for linha in conteudo.splitlines():
            if any(p in linha.lower() for p in pergunta_lower.split()):
                trecho_encontrado = linha.strip()
                break
        if not trecho_encontrado:
            trecho_encontrado = textwrap.shorten(conteudo.replace("\n", " "), width=300, placeholder=" [...]")

        link = encontrar_link(nome)
        similaridade = round(np.dot(index.reconstruct(idx), embedding[0]), 2)

        resultados.append({
            "nome": nome,
            "trecho": trecho_encontrado,
            "link": link,
            "similaridade": similaridade
        })

    return resultados

# 🔁 Loop de busca interativo
while True:
    pergunta = input("📝 Digite sua pergunta (ou 'sair' para encerrar): ")
    if pergunta.lower().strip() in ["sair", "exit", "quit"]:
        print("👋 Encerrando a busca. Até logo!")
        break

    # 1. Tenta busca literal primeiro
    resultados = busca_literal_em_todos(pergunta)

    # 2. Se não encontrar nada, faz busca semântica
    if not resultados:
        resultados = busca_semantica(pergunta)

    if not resultados:
        print("⚠️ Nenhum documento encontrado.")
        continue

    print(f"\n📚 Encontrei {len(resultados)} documento(s) relevante(s):\n")

    for i, doc in enumerate(resultados, 1):
        print(f"{i}. 📄 {doc['nome']}")
        if doc["similaridade"] is not None:
            print(f"🔍 Similaridade: {doc['similaridade']}")
        print(f"🧾 Trecho relevante:\n\n{doc['trecho']}\n")
        if doc["link"]:
            display(Markdown(f"🔗 [Clique para abrir o PDF original]({doc['link']})"))
        else:
            print("⚠️ Link não disponível.")
        print("\n" + "-"*80)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.0 MB/s eta 0:00:00
📝 Digite sua pergunta (ou 'sair' para encerrar): Programa de intercâmbio

📚 Encontrei 2 documento(s) relevante(s):

1. 📄 Resolução_0132014_-_CONSEPE_-_Aprova_o_Regimento_Geral_da_Pós-Graduação_Stricto_Sensu_da_UDESC.txt
🧾 Trecho relevante:

§ 2º A incorporação dos alunos no programa de intercâmbio sujeitar-se-á às regras



🔗 [Clique para abrir o PDF original](http://secon.udesc.br/consepe/resol/2014/013-2014-cpe.pdf)


--------------------------------------------------------------------------------
2. 📄 Resolução_0072017_-_CONSEPE_-_Altera_dispositivos_do_Regimento_Geral_da_Pós-Graduação_Stricto_Sensu_da_UDESC.txt
🧾 Trecho relevante:

§ 2º A incorporação dos alunos no programa de intercâmbio sujeitar-se-á às



🔗 [Clique para abrir o PDF original](http://www.udesc.br/arquivos/cct/id_cpmenu/1017/resolucao_007_2017_consepe_15166515275844_1017.pdf)


--------------------------------------------------------------------------------
📝 Digite sua pergunta (ou 'sair' para encerrar): Docente permanente

📚 Encontrei 7 documento(s) relevante(s):

1. 📄 Plano_atual_curso_MCAPPGCAP_Julho2022.txt
🧾 Trecho relevante:

um docente permanente do programa, foram adquiridos alguns equipamentos



...


--------------------------------------------------------------------------------
6. 📄 Portaria_0812016_-_CAPES.txt
🧾 Trecho relevante:

Art. 4º A atuação como docente permanente poderá se dar,



🔗 [Clique para abrir o PDF original](http://www.udesc.br/arquivos/cct/id_cpmenu/1017/Portaria_8_2016_de_03_06_2016_CAPES_15263035171951_1017.pdf)


--------------------------------------------------------------------------------
7. 📄 Resolução_0132014_-_CONSEPE_-_Aprova_o_Regimento_Geral_da_Pós-Graduação_Stricto_Sensu_da_UDESC.txt
🧾 Trecho relevante:

“Art. 65A. A atuação como docente permanente poderá se dar, no máximo, em até 3 (três)



🔗 [Clique para abrir o PDF original](http://secon.udesc.br/consepe/resol/2014/013-2014-cpe.pdf)


--------------------------------------------------------------------------------
📝 Digite sua pergunta (ou 'sair' para encerrar): sair
👋 Encerrando a busca. Até logo!
